# Selection Widget Manual Test

This notebook provides a manual test workflow for the SelectionWidget.

It creates a small test database with known data, so results can be verified.

## Test Workflow
1. Create a test database with sample device data
2. Create a new selection project
3. Add a group with keywords
4. Complete the selection workflow (all 3 phases)
5. Verify results are correct
6. Test persistence across kernel restart

In [ ]:
import sys
import os
import sqlite3
from pathlib import Path

# Add src to path for development
src_path = Path().resolve().parent.parent / 'src'
sys.path.insert(0, str(src_path))

from pymaude import MaudeDatabase, SelectionManager
from pymaude.selection_widget import SelectionWidget

## Step 1: Create Test Database

Create a small SQLite database with known test data.

In [ ]:
TEST_DB_PATH = './test_widget.db'
TEST_SELECTION_PATH = './test_widget_selection.json'

# Remove existing test files
for f in [TEST_DB_PATH, TEST_SELECTION_PATH]:
    if os.path.exists(f):
        os.remove(f)

# Create test database with sample data
conn = sqlite3.connect(TEST_DB_PATH)
cursor = conn.cursor()

# Create device table
cursor.execute('''
    CREATE TABLE device (
        MDR_REPORT_KEY INTEGER,
        BRAND_NAME TEXT,
        GENERIC_NAME TEXT,
        MANUFACTURER_D_NAME TEXT,
        DATE_RECEIVED TEXT,
        DEVICE_REPORT_PRODUCT_CODE TEXT
    )
''')

# Create master table
cursor.execute('''
    CREATE TABLE master (
        MDR_REPORT_KEY INTEGER,
        EVENT_KEY INTEGER,
        DATE_RECEIVED TEXT,
        EVENT_TYPE TEXT
    )
''')

# Insert test device data
# Group 1: Penumbra devices (should match 'penumbra' keyword)
test_devices = [
    # Penumbra Lightning devices - 5 records
    (1001, 'PENUMBRA LIGHTNING BOLT 7', 'THROMBECTOMY CATHETER', 'PENUMBRA INC', '2023-01-15', 'NIQ'),
    (1002, 'PENUMBRA LIGHTNING BOLT 7', 'THROMBECTOMY CATHETER', 'PENUMBRA INC', '2023-02-20', 'NIQ'),
    (1003, 'PENUMBRA LIGHTNING FLASH', 'THROMBECTOMY CATHETER', 'PENUMBRA INC', '2023-03-10', 'NIQ'),
    (1004, 'PENUMBRA INDIGO SYSTEM', 'ASPIRATION CATHETER', 'PENUMBRA INC', '2023-04-05', 'NIQ'),
    (1005, 'PENUMBRA', 'CATHETER', 'PENUMBRA INC', '2023-05-01', 'NIQ'),  # Generic name only
    
    # Penumbra non-thrombectomy (should be rejectable)
    (1006, 'PENUMBRA SMART COIL', 'EMBOLIZATION COIL', 'PENUMBRA INC', '2023-06-15', 'MBP'),
    (1007, 'PENUMBRA RUBY COIL', 'EMBOLIZATION COIL', 'PENUMBRA INC', '2023-07-20', 'MBP'),
    
    # Inari devices (separate group test)
    (2001, 'INARI FLOWTRIEVER', 'THROMBECTOMY DEVICE', 'INARI MEDICAL', '2023-01-10', 'NIQ'),
    (2002, 'INARI CLOTTRIEVER', 'THROMBECTOMY DEVICE', 'INARI MEDICAL', '2023-02-15', 'NIQ'),
    (2003, 'FLOWTRIEVER T24', 'MECHANICAL THROMBECTOMY', 'INARI MEDICAL INC', '2023-03-20', 'NIQ'),
    
    # Unrelated devices (should not match)
    (3001, 'SOME OTHER DEVICE', 'PACEMAKER', 'OTHER COMPANY', '2023-01-01', 'XXX'),
    (3002, 'ANOTHER DEVICE', 'STENT', 'ANOTHER COMPANY', '2023-02-01', 'YYY'),
]

cursor.executemany(
    'INSERT INTO device VALUES (?, ?, ?, ?, ?, ?)',
    test_devices
)

# Insert corresponding master records
test_master = [
    (1001, 1001, '2023-01-15', 'M'),
    (1002, 1002, '2023-02-20', 'IN'),
    (1003, 1003, '2023-03-10', 'M'),
    (1004, 1004, '2023-04-05', 'M'),
    (1005, 1005, '2023-05-01', 'M'),
    (1006, 1006, '2023-06-15', 'M'),
    (1007, 1007, '2023-07-20', 'M'),
    (2001, 2001, '2023-01-10', 'D'),
    (2002, 2002, '2023-02-15', 'IN'),
    (2003, 2003, '2023-03-20', 'M'),
    (3001, 3001, '2023-01-01', 'M'),
    (3002, 3002, '2023-02-01', 'M'),
]

cursor.executemany(
    'INSERT INTO master VALUES (?, ?, ?, ?)',
    test_master
)

# Create indexes
cursor.execute('CREATE INDEX idx_device_key ON device(MDR_REPORT_KEY)')
cursor.execute('CREATE INDEX idx_device_brand ON device(BRAND_NAME)')
cursor.execute('CREATE INDEX idx_master_key ON master(MDR_REPORT_KEY)')

conn.commit()
conn.close()

print(f"Created test database with {len(test_devices)} device records")
print("\nExpected results for 'penumbra' keyword:")
print("  - 7 matching device records (MDRs 1001-1007)")
print("  - Brand names: PENUMBRA LIGHTNING BOLT 7, PENUMBRA LIGHTNING FLASH, PENUMBRA INDIGO SYSTEM, PENUMBRA, PENUMBRA SMART COIL, PENUMBRA RUBY COIL")
print("\nExpected results for 'inari' keyword:")
print("  - 3 matching device records (MDRs 2001-2003)")

## Step 2: Connect to Test Database

In [ ]:
db = MaudeDatabase(TEST_DB_PATH, verbose=False)

# Verify data
device_count = db.query("SELECT COUNT(*) as cnt FROM device")['cnt'].iloc[0]
master_count = db.query("SELECT COUNT(*) as cnt FROM master")['cnt'].iloc[0]
print(f"Database has {device_count} device records and {master_count} master records")

## Step 3: Create Selection Manager

In [ ]:
manager = SelectionManager('widget_test', TEST_SELECTION_PATH, db.db_path)
print(f"Created selection manager: {manager.name}")

## Step 4: Test Search Preview

Verify that search preview returns expected counts.

In [ ]:
# Test search preview for 'penumbra'
preview = manager.get_search_preview(db, ['penumbra'])
print("Search preview for 'penumbra':")
print(f"  Brand names: {preview['brand_name_count']} unique values, {preview['brand_name_mdrs']} MDRs")
print(f"  Generic names: {preview['generic_name_count']} unique values")
print(f"  Manufacturers: {preview['manufacturer_count']} unique values")
print(f"  Total unique MDRs: {preview['total_unique_mdrs']}")

# Assertions
assert preview['total_unique_mdrs'] == 7, f"Expected 7 MDRs, got {preview['total_unique_mdrs']}"
print("\n[PASS] Search preview returns correct counts")

## Step 5: Launch Widget

**Manual Test Instructions:**

1. Click **+ Add New Group**
2. Enter keywords: `penumbra`
3. Click **Search Preview** to see counts
4. Enter group name: `penumbra_test`
5. Click **Proceed to Selection**

**In Brand Name Review:**
- **Accept**: PENUMBRA LIGHTNING BOLT 7, PENUMBRA LIGHTNING FLASH, PENUMBRA INDIGO SYSTEM
- **Defer**: PENUMBRA (too generic)
- **Reject**: PENUMBRA SMART COIL, PENUMBRA RUBY COIL (not thrombectomy)

6. Click **Next Phase** (Generic Name)
7. You should only see the deferred 'PENUMBRA' MDR now
8. Accept or reject based on generic name
9. Continue to Manufacturer phase
10. Click **Finalize Group**

In [ ]:
widget = SelectionWidget(manager, db)
widget.display()

## Step 6: Verify Results After Widget Interaction

Run this cell after completing the widget workflow above.

In [ ]:
# Reload manager to ensure we have saved state
manager = SelectionManager.load(TEST_SELECTION_PATH)

print("Groups in manager:")
for group_name in manager.groups:
    status = manager.get_group_status(group_name)
    print(f"  {group_name}: status={status['status']}, phase={status['current_phase']}")
    print(f"    Decisions: {status['decisions_count']}")

In [ ]:
# Get results
if manager.groups:
    results = manager.get_results(db, mode='decisions')
    print("\nResults summary:")
    print(results.summary)
    
    # Show data for each group
    for group_name in results.groups:
        df = results[group_name]
        print(f"\n{group_name}: {len(df)} records")
        if len(df) > 0:
            print(df[['MDR_REPORT_KEY', 'BRAND_NAME', 'EVENT_TYPE']].head(10))
else:
    print("No groups defined yet. Complete the widget workflow first.")

## Step 7: Test Programmatic Workflow

This tests the SelectionManager without the widget.

In [ ]:
# Create a fresh manager for programmatic test
PROG_SELECTION_PATH = './test_programmatic_selection.json'
if os.path.exists(PROG_SELECTION_PATH):
    os.remove(PROG_SELECTION_PATH)

prog_manager = SelectionManager('programmatic_test', PROG_SELECTION_PATH, db.db_path)

# Create group
prog_manager.create_group('inari', ['inari', 'flowtriever', 'clottriever'])

# Search candidates
candidates = prog_manager.search_candidates(db, 'inari', 'brand_name')
print("Brand name candidates for 'inari':")
print(candidates)

# Accept all
for value in candidates['value']:
    prog_manager.set_decision('inari', 'brand_name', value, 'accept')

# Advance through phases
prog_manager.advance_phase('inari')  # to generic_name
prog_manager.advance_phase('inari')  # to manufacturer
prog_manager.advance_phase('inari')  # to finalized

# Finalize
result = prog_manager.finalize_group(db, 'inari')
print(f"\nFinalized: {result['mdr_count']} MDRs")

# Verify
assert result['mdr_count'] == 3, f"Expected 3 MDRs, got {result['mdr_count']}"
print("[PASS] Programmatic workflow completed correctly")

# Get results
prog_results = prog_manager.get_results(db)
print(f"\nInari group has {len(prog_results['inari'])} records")
print(prog_results['inari'][['MDR_REPORT_KEY', 'BRAND_NAME', 'EVENT_TYPE']])

## Step 8: Test Persistence

Verify that state persists correctly.

In [ ]:
# Save
prog_manager.save()

# Reload
reloaded = SelectionManager.load(PROG_SELECTION_PATH)

# Verify state
assert 'inari' in reloaded.groups
assert reloaded.groups['inari']['status'] == 'complete'
assert reloaded.groups['inari']['mdr_keys_snapshot'] is not None
assert len(reloaded.groups['inari']['mdr_keys_snapshot']) == 3

print("[PASS] Persistence test passed")
print(f"Reloaded manager has groups: {list(reloaded.groups.keys())}")
print(f"MDR keys snapshot: {reloaded.groups['inari']['mdr_keys_snapshot']}")

## Cleanup

In [ ]:
db.close()

# Remove test files
for f in [TEST_DB_PATH, TEST_SELECTION_PATH, PROG_SELECTION_PATH]:
    if os.path.exists(f):
        os.remove(f)
        print(f"Removed: {f}")

print("\nCleanup complete!")